In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#install libs
!pip install ultralytics torch torchvision pillow opencv-python pandas


In [ ]:
import os
import time

import torch
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn

from ultralytics import YOLO

from PIL import Image
import cv2
import numpy as np
import pandas as pd


In [ ]:
#load yolo model
yolo_model = YOLO('yolov8n.pt')

#load faster r-cnn model with pretrained weights
faster_rcnn = fasterrcnn_resnet50_fpn(weights="DEFAULT")
faster_rcnn.eval()  #set to eval mode

#put models on device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
faster_rcnn.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
image_dir = "/content/drive/MyDrive/MSBA503_CV_Project/Images"
os.makedirs(image_dir, exist_ok=True)
image_paths = [os.path.join(image_dir, f)
               for f in os.listdir(image_dir)
               if f.lower().endswith('.jpg')]

image_paths = sorted(image_paths)
image_paths

['/content/drive/MyDrive/MSBA503_CV_Project/Images/img01.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img02.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img03.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img04.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img05.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img06.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img07.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img08.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img09.jpg',
 '/content/drive/MyDrive/MSBA503_CV_Project/Images/img10.jpg']

In [ ]:
 #run yolo on 1 image
def run_yolo(image_path, conf_threshold=0.5):
    start = time.time()
    results = yolo_model(image_path, verbose=False)
    end = time.time()

    #results[0]
    boxes = results[0].boxes
    scores = boxes.conf.cpu().numpy()

    #filter by confidence
    keep = scores >= conf_threshold
    scores_filtered = scores[keep]

    num_objects = len(scores_filtered)
    avg_conf = float(scores_filtered.mean()) if num_objects > 0 else 0.0
    elapsed = end - start

    return num_objects, avg_conf, elapsed


In [ ]:
#run faster r-cnn on 1 image
def run_faster_rcnn(image_path, conf_threshold=0.5):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.ToTensor()])
    img_tensor = transform(image).to(device)

    start = time.time()
    with torch.no_grad():
        outputs = faster_rcnn([img_tensor])
    end = time.time()

    output = outputs[0]
    scores = output["scores"].cpu().numpy()

    keep = scores >= conf_threshold
    scores_filtered = scores[keep]

    num_objects = len(scores_filtered)
    avg_conf = float(scores_filtered.mean()) if num_objects > 0 else 0.0
    elapsed = end - start

    return num_objects, avg_conf, elapsed


In [ ]:
results_list = []

for img_path in image_paths:
    img_name = os.path.basename(img_path)

    #run yolo
    yolo_num, yolo_avg_conf, yolo_time = run_yolo(img_path)
    results_list.append({
        "image": img_name,
        "model": "YOLOv8n",
        "num_objects": yolo_num,
        "avg_confidence": yolo_avg_conf,
        "time_sec": yolo_time
    })

    #run faster r-cnn
    frcnn_num, frcnn_avg_conf, frcnn_time = run_faster_rcnn(img_path)
    results_list.append({
        "image": img_name,
        "model": "Faster R-CNN",
        "num_objects": frcnn_num,
        "avg_confidence": frcnn_avg_conf,
        "time_sec": frcnn_time
    })

#create df
df_results = pd.DataFrame(results_list)
df_results


,image,model,num_objects,avg_confidence,time_sec
0,img01.jpg,YOLOv8n,1,0.689559,0.275396
1,img01.jpg,Faster R-CNN,1,0.798524,6.322358
2,img02.jpg,YOLOv8n,1,0.913346,0.507425
3,img02.jpg,Faster R-CNN,2,0.978420,8.541780
4,img03.jpg,YOLOv8n,2,0.624771,0.184349
5,img03.jpg,Faster R-CNN,0,0.000000,8.393308
6,img04.jpg,YOLOv8n,1,0.543403,0.166904
7,img04.jpg,Faster R-CNN,2,0.680712,8.368860
8,img05.jpg,YOLOv8n,1,0.725875,0.235572
9,img05.jpg,Faster R-CNN,0,0.000000,7.787477


In [ ]:
summary = df_results.groupby("model").agg({
    "num_objects": "mean",
    "avg_confidence": "mean",
    "time_sec": "mean"
}).reset_index()

summary_rounded = summary.round(3)
summary_rounded


,model,num_objects,avg_confidence,time_sec
0,Faster R-CNN,1.6,0.479,7.787
1,YOLOv8n,0.8,0.530,0.250


In [ ]:
def get_avg_color(image_path):
    # read image with cv2
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #average over all pixels
    avg_color = img.mean(axis=(0, 1))  # (R, G, B)
    r, g, b = avg_color
    return float(r), float(g), float(b) #red, green, blue

color_rows = []

for img_path in image_paths:
    img_name = os.path.basename(img_path)
    r, g, b = get_avg_color(img_path)
    color_rows.append({
        "image": img_name,
        "avg_red": r,
        "avg_green": g,
        "avg_blue": b
    })

df_colors = pd.DataFrame(color_rows)
df_colors


,image,avg_red,avg_green,avg_blue
0,img01.jpg,203.069748,172.856849,149.598222
1,img02.jpg,175.731639,172.372430,171.566970
2,img03.jpg,102.900156,123.137874,131.583101
3,img04.jpg,228.017308,227.837818,228.474278
4,img05.jpg,231.456274,218.393840,205.020566
5,img06.jpg,246.256660,219.186457,175.585586
6,img07.jpg,233.451861,234.836438,237.263581
7,img08.jpg,163.380397,162.448262,161.854697
8,img09.jpg,85.058397,148.826956,175.065442
9,img10.jpg,207.835558,213.007973,226.853552
